## Standard Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import r2_score

# impute missing values
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from category_encoders import TargetEncoder


#Some pandas extra options
pd.options.mode.use_inf_as_na = True
pd.set_option('display.max_columns',50)

In [4]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer,
    get_package_path, display_as_tabs, save_pipeline, load_pipeline, initialize_environment,
    load_dataset, save_dataset, DEFAULT_ARTIFACTS_PATH
)

import ta_lib.eda.api as eda
from xgboost import XGBRegressor
from ta_lib.regression.api import SKLStatsmodelOLS
from ta_lib.regression.api import RegressionComparison, RegressionReport
import ta_lib.reports.api as reports
from ta_lib.data_processing.api import Outlier, FeatureSelectorStatistic, FeatureSelector


initialize_environment(debug=False, hide_warnings=True)

## Initialization

In [5]:
artifacts_folder = DEFAULT_ARTIFACTS_PATH

In [6]:
config_path = op.join('conf','config.yml')
context = create_context(config_path)

# 1. Feature Engineering

## Loading the test and train data

In [52]:
X_train = load_dataset(context, 'train/sales/features')
X_test = load_dataset(context, 'test/sales/features')
print(X_train.shape, X_test.shape)

y_train = load_dataset(context, 'train/sales/target')
y_test = load_dataset(context, 'test/sales/target')
print(y_train.shape, y_test.shape)

(4994411, 11) (1248603, 11)
(4994411, 1) (1248603, 1)


## 1.1 Feature Engineering Pipelines

#### General Steps in the Feature Transformation are as follows
 - Outlier Treatment
 - Encoding of Categorical Columns
 - Missing Values Imputation
     - There are no missing values to impute.

### 1.1.1 Outlier Treatment

In [8]:
eda.get_outliers(X_train)

Data Shape:(4994411, 11)                 \
feature                     < (mean-3*std) > (mean+3*std)   
sales_units_value                        0          49756   
sales_lbs_value                          0          48721   
sales_year                               0              0   
total_post                               0         104453   
search_volume                            0          57879   

                                                                       
feature           < (1stQ - 1.5 * IQR) > (3rdQ + 1.5 * IQR) -inf +inf  
sales_units_value                    0               588579    0    0  
sales_lbs_value                      0               695342    0    0  
sales_year                      462908                    0    0    0  
total_post                           0               307283    0    0  
search_volume                        0                60317    0    0

In [53]:
outlier_transform = Outlier()
X_train = outlier_transform.fit_transform(X_train,
                                          cols=['sales_units_value', 'sales_lbs_value', 'total_post', 'search_volume'])

In [10]:
eda.get_outliers(X_train)

Data Shape:(4994411, 11)                 \
feature                     < (mean-3*std) > (mean+3*std)   
sales_units_value                        0         134564   
sales_lbs_value                          0         120167   
sales_year                               0              0   
total_post                               0         104551   
search_volume                            0          69296   

                                                                       
feature           < (1stQ - 1.5 * IQR) > (3rdQ + 1.5 * IQR) -inf +inf  
sales_units_value                    0               588579    0    0  
sales_lbs_value                      0               695342    0    0  
sales_year                      462908                    0    0    0  
total_post                           0               307283    0    0  
search_volume                        0                60317    0    0

### 1.1.2 Encoding of Categorical Columns

In [54]:
feature_transformer = make_column_transformer(
    
        #Too many levels
        (TargetEncoder(return_df=False), ['theme']),
    
        #Sparse levels
        (OneHotEncoder(sparse=False), ['vendor','platform']),
    
        #Scaling
#         (MinMaxScaler(), ['sales_units_value','sales_lbs_value', 'total_post', 'search_volume', 'sales_week_no']),
    
        remainder='passthrough'
    
)

In [55]:
X_train_transformed = get_dataframe(
    feature_transformer.fit_transform(X_train, y_train),
    get_feature_names_from_column_transformer(feature_transformer)
)

In [13]:
# save the feature pipeline
save_pipeline(feature_transformer, op.abspath(op.join(artifacts_folder, 'features.joblib')))

#save the scaled version of X_train
save_dataset(context, X_train_transformed, 'train/sales/transformed_features')

## 1.2 Feature Analysis

### 1.2.1 Univariate Analysis
* Understanding each variable seperately.

In [ ]:
%%time
out = eda.get_density_plots(X_train_transformed)
# out

#### Creating a .html report containing 
* density plots

In [ ]:
%%time
reports.create_report({'univariate': out}, name='feature_analysis_univariate')

#### Creating a .html report containing 
* Summary Stats
* distributions(density plot and percentile plots)
* feature normality

In [ ]:
%%time
reports.feature_analysis(X_train_transformed, './feature_analysis_report.html')

### 1.2.2 Bivariate Analysis
* Understanding the mutual interaction between the features (Correlations b/w features)
* Setting the threshold to 0.6 and removing all the highly correlated features.

In [14]:
out = eda.get_correlation_table(X_train_transformed)
out[out['Abs Corr Coef'] > 0.6]

,Variable 1,Variable 2,Corr Coef,Abs Corr Coef
0,sales_month,sales_week_no,0.996062,0.996062
1,sales_month,sales_quarter,0.967506,0.967506
2,sales_quarter,sales_week_no,0.964193,0.964193
3,platform_google,search_volume,0.709206,0.709206


In [56]:
curated_cols = list( 
    set(X_train_transformed.columns) - 
    set(['sales_month', 'sales_quarter', 'platform_google'])
)
X_train_transformed = X_train_transformed[curated_cols]

In [16]:
out = eda.get_correlation_table(X_train_transformed)
out[out['Abs Corr Coef'] > 0.6]

,Variable 1,Variable 2,Corr Coef,Abs Corr Coef


# 3. Modelling

## 3.1 Linear Regressor

### 3.1.1 Feature Selection
* Done on the basis of VIF score and manual selection in an iterative process.

In [17]:
# To avoid the dummy variable trap, we exclude one feature from each of the onehotencoded features
eda.calc_vif(X_train_transformed.drop('vendor_Others', axis=1))

,variables,VIF
0,theme,6.516775
1,sales_year,19.879829
2,search_volume,4.188514
3,vendor_Private Label,1.142252
4,total_post,3.076764
5,vendor_D,1.086785
6,platform_amazon,2.428934
7,vendor_B,1.121030
8,sales_lbs_value,1.396071
9,sales_units_value,1.530015


In [18]:
# cols are selected through manual selection
cols = ['sales_year']

In [19]:
# cols are selected through manual selection
cols = ['sales_year']
eda.calc_vif(X_train_transformed.drop(cols, axis=1).drop('vendor_Others', axis=1))

,variables,VIF
0,theme,3.862846
1,search_volume,2.847386
2,vendor_Private Label,1.141567
3,total_post,2.988258
4,vendor_D,1.086565
5,platform_amazon,1.817687
6,vendor_B,1.116708
7,sales_lbs_value,1.394258
8,sales_units_value,1.524941
9,platform_walmart,1.997411


### 3.1.2 Data transformation

In [20]:
def _custom_data_transform(df, cols2remove=None):
    """Transformation to drop some columns in the data
    
    Parameters
    ----------
        df - pd.DataFrame
        cols2remove - columns to keep in the dataframe
    """
    cols2remove = cols2remove or []
    if len(cols2remove):
        return (df
                .drop(cols2remove, axis=1))
    else:
        return df

### 3.1.3 Model Training Pipeline (Linear Regression)

In [21]:
outlier_cols = ['sales_units_value', 'sales_lbs_value', 'total_post', 'search_volume']
columns = list(X_train_transformed.columns)

reg_ols_ppln = Pipeline([
    ('outlier', Outlier()),
    ('', FunctionTransformer(_custom_data_transform, kw_args={'cols2remove':cols})),
    ('estimator', SKLStatsmodelOLS())
])

reg_ols_ppln.fit(X_train_transformed, y_train.values.ravel(), outlier__cols=outlier_cols)

Pipeline(steps=[('outlier',
                 <tigerml.core.preprocessing.outliers.Outlier object at 0x000002BC8008D430>),
                ('',
                 FunctionTransformer(func=<function _custom_data_transform at 0x000002BCD9F8FF70>,
                                     kw_args={'cols2remove': ['sales_year']})),
                ('estimator', SKLStatsmodelOLS())])

In [22]:
reg_ols_ppln['estimator'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.713
Method:                 Least Squares   F-statistic:                 8.259e+05
Date:                Mon, 23 May 2022   Prob (F-statistic):               0.00
Time:                        09:35:48   Log-Likelihood:            -5.6855e+07
No. Observations:             4994411   AIC:                         1.137e+08
Df Residuals:                 4994395   BIC:                         1.137e+08
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
intercept            -2423.9020     43.441    -55.798      0.000   -2509.045   -2338.759
vendor_Others         1544.3648     21.114     73.146      0.000    1502.983    1585.747
theme                    0.0767      0.002     44.514      0.000       0.073       0.080
search_volume           -0.0117      0.001    -17.561      0.000      -0.013      -0.010
vendor_Private Label -3854.7402     33.625   -114.638      0.000   -3920.645   -3788.836
total_post               0.0629      0.006     10.391      0.000       0.051       0.075
vendor_D               718.0479     49.191     14.597      0.000     621.635     814.461
platform_amazon       -121.1460     32.794     -3.694      0.000    -185.421     -56.871
vendor_B             -1831.3890     42.522    -43.069      0.000   -1914.731   -1748.047
sales_lbs_value          2.5764      0.001   2825.224      0.000       2.575       2.578
sales_units_value        2.3456      0.004    648.493      0.000       2.338       2.353
platform_walmart      -484.5313     37.277    -12.998      0.000    -557.594    -411.469
platform_chewy        -319.6548     36.750     -8.698      0.000    -391.684    -247.626
vendor_A              -106.0026     32.461     -3.266      0.001    -169.625     -42.381
vendor_F              2703.0429     38.123     70.903      0.000    2628.323    2777.762
sales_week_no            2.4595      0.679      3.623      0.000       1.129       3.790
vendor_H             -1597.2257     97.441    -16.392      0.000   -1788.207   -1406.244
==============================================================================
Omnibus:                  9100518.647   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      25882056885.777
Skew:                          13.262   Prob(JB):                         0.00
Kurtosis:                     354.667   Cond. No.                     8.59e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.4e-19. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [57]:
X_test_transformed = get_dataframe(
    feature_transformer.transform(X_test),
    get_feature_names_from_column_transformer(feature_transformer)
)
X_test_transformed = X_test_transformed[curated_cols]

#### Train R2 score

In [82]:
yhat_train_ols = reg_ols_ppln.predict(X_train_transformed)
r2_score(y_train, yhat_train_ols)

0.712681136683002

#### Test R2 score

In [83]:
yhat_test_ols = reg_ols_ppln.predict(X_test_transformed)
r2_score(y_test, yhat_test_ols)

0.7129558393451261

In [26]:
save_pipeline(reg_ols_ppln, op.abspath(op.join(artifacts_folder, 'reg_ols.joblib')))

In [ ]:
# reg_ols_ppln = load_pipeline(op.abspath(op.join(artifacts_folder, 'reg_ols.joblib')))

### 3.1.4 Model Evaluation Report (Linear Regression)

In [111]:
%%time
reg_linear_report = RegressionReport(x_train=X_train_transformed, y_train=y_train, x_test=X_test_transformed, y_test=y_test, yhat_train=yhat_train_ols, yhat_test=yhat_test_ols)
reg_linear_report.get_report(include_shap=False, file_path='EvalReports/regression_linear_model_report')

Wall time: 6.93 s


## 3.2 XGBoost

### 3.2.1 Model Training Pipeline (XGBoost)

### Model-1

In [28]:
%%time
xgb_ppln_init = Pipeline([
    ('XGBoost', XGBRegressor(tree_method='approx', n_jobs=4))
])
xgb_ppln_init.fit(X_train_transformed, y_train.values.ravel())

Wall time: 7min 24s


Pipeline(steps=[('XGBoost',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=4, num_parallel_tree=1, random_state=0,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='approx',
                              validate_parameters=1, verbosity=None))])

#### Train R2 score

In [84]:
yhat_train_xgb = xgb_ppln_init.predict(X_train_transformed)
r2_score(y_train, yhat_train_xgb)

0.8958516920748202

#### Test R2 score

In [85]:
yhat_test_xgb = xgb_ppln_init.predict(X_test_transformed)
r2_score(y_test, yhat_test_xgb)

0.8897438929100367

### Model-2

In [91]:
%%time
xgb_ppln_init = Pipeline([
    ('XGBoost', XGBRegressor(tree_method='exact', n_jobs=4))
])
xgb_ppln_init.fit(X_train_transformed, y_train.values.ravel())

Wall time: 6min


Pipeline(steps=[('XGBoost',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=4, num_parallel_tree=1, random_state=0,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity=None))])

#### Train R2 score

In [92]:
yhat_train_xgb = xgb_ppln_init.predict(X_train_transformed)
r2_score(y_train, yhat_train_xgb)

0.9166466513208926

#### Test R2 score

In [93]:
yhat_test_xgb = xgb_ppln_init.predict(X_test_transformed)
r2_score(y_test, yhat_test_xgb)

0.911431576787131

### Model-3

In [94]:
%%time
xgb_ppln_init = Pipeline([
    ('XGBoost', XGBRegressor(tree_method='exact', max_depth=10, n_jobs=4))
])
xgb_ppln_init.fit(X_train_transformed, y_train.values.ravel())

Wall time: 11min 7s


Pipeline(steps=[('XGBoost',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=10, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=4, num_parallel_tree=1, random_state=0,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity=None))])

#### Train R2 score

In [95]:
yhat_train_xgb = xgb_ppln_init.predict(X_train_transformed)
r2_score(y_train, yhat_train_xgb)

0.9420353696117304

#### Test R2 score

In [96]:
yhat_test_xgb = xgb_ppln_init.predict(X_test_transformed)
r2_score(y_test, yhat_test_xgb)

0.9251769862722912

### Model-4

In [97]:
%%time
xgb_ppln_init = Pipeline([
    ('XGBoost', XGBRegressor(tree_method='exact', max_depth=10, n_estimators=300, n_jobs=4))
])
xgb_ppln_init.fit(X_train_transformed, y_train.values.ravel())

Wall time: 31min 5s


Pipeline(steps=[('XGBoost',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=10, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=300,
                              n_jobs=4, num_parallel_tree=1, random_state=0,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity=None))])

#### Train R2 score

In [98]:
yhat_train_xgb = xgb_ppln_init.predict(X_train_transformed)
r2_score(y_train, yhat_train_xgb)

0.949942307941203

#### Test R2 score

In [99]:
yhat_test_xgb = xgb_ppln_init.predict(X_test_transformed)
r2_score(y_test, yhat_test_xgb)

0.9279208759213714

In [105]:
save_pipeline(xgb_ppln_init, op.abspath(op.join(artifacts_folder, 'xgb_init.joblib')))

In [106]:
%%time
reg_linear_report = RegressionReport(x_train=X_train_transformed, y_train=y_train, x_test=X_test_transformed, y_test=y_test, yhat_train=yhat_train_xgb, yhat_test=yhat_test_xgb)
reg_linear_report.get_report(include_shap=True, file_path='EvalReports/regression_xgb_init_shap_report')

Wall time: 5.36 s


In [16]:
# xgb_ppln_init = load_pipeline(op.abspath(op.join(artifacts_folder, 'xgb_init.joblib')))

## 3.3 Random Forest

In [71]:
from sklearn.ensemble import RandomForestRegressor

### 3.3.1 Model-1

In [72]:
%%time
rf_ppln_init = Pipeline([
    ('rf', RandomForestRegressor(max_depth=7, n_jobs=4))
    
])

rf_ppln_init.fit(X_train_transformed, y_train.values.ravel())

Wall time: 11min 59s


Pipeline(steps=[('rf', RandomForestRegressor(max_depth=7, n_jobs=4))])

#### Train R2 score

In [86]:
yhat_train_rf = rf_ppln_init.predict(X_train_transformed)
r2_score(y_train, yhat_train_rf)

0.8737390082952861

#### Test R2 score

In [87]:
yhat_test_rf = rf_ppln_init.predict(X_test_transformed)
r2_score(y_test, yhat_test_rf)

0.8744110519509574

### 3.3.2 Model-2

In [100]:
%%time
rf_ppln_init = Pipeline([
    ('rf', RandomForestRegressor(max_depth=10, n_estimators=300, n_jobs=4))
    
])

rf_ppln_init.fit(X_train_transformed, y_train.values.ravel())

Wall time: 50min 31s


Pipeline(steps=[('rf',
                 RandomForestRegressor(max_depth=10, n_estimators=300,
                                       n_jobs=4))])

#### Train R2 score

In [101]:
yhat_train_rf = rf_ppln_init.predict(X_train_transformed)
r2_score(y_train, yhat_train_rf)

0.9036879800832648

#### Test R2 score

In [102]:
yhat_test_rf = rf_ppln_init.predict(X_test_transformed)
r2_score(y_test, yhat_test_rf)

0.8998393422423504

In [103]:
save_pipeline(rf_ppln_init, op.abspath(op.join(artifacts_folder, 'rf_ppln_init.joblib')))

In [107]:
%%time
reg_linear_report = RegressionReport(x_train=X_train_transformed, y_train=y_train, x_test=X_test_transformed, y_test=y_test, yhat_train=yhat_train_rf, yhat_test=yhat_test_rf)
reg_linear_report.get_report(include_shap=True, file_path='EvalReports/regression_rf_init_shap_report')

Wall time: 5.3 s


In [ ]:
# rf_ppln_init = load_pipeline(op.abspath(op.join(artifacts_folder, 'rf_ppln_init.joblib')))

## 4. Model Comparison

In [109]:
%%time
model_comparison_report = RegressionComparison(y=y_test, yhats={'Linear Regression': yhat_test_ols, 'XGBoost': yhat_test_xgb, 'Random Forest': yhat_test_rf})
metrics = model_comparison_report.get_report(file_path='EvalReports/regression_comparison')

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Wall time: 19.1 s


In [110]:
model_comparison_report.performance_metrics

metric,MAPE,WMAPE,MAE,RMSE,R^2
Linear Regression,20.4722,0.4726,6016.6827,21236.0083,0.7130
XGBoost,7.2635,0.1832,2331.8622,10641.5105,0.9279
Random Forest,8.5890,0.2678,3409.1509,12544.3197,0.8998


In [66]:
# %%time
# 
# xgb_ppln_2 = Pipeline([
# #     ('',FunctionTransformer(_custom_data_transform, kw_args={'cols2remove':cols_2_remove})),
#     ('XGBoost', XGBRegressor(tree_method='approx', n_estimators=300, n_jobs=4))
# ])
# eval_set = [(X_train_transformed, y_train.values.ravel()), 
#             (X_test_transformed, y_test.values.ravel())]
# 
# xgb_ppln_2.fit(X_train_transformed, y_train.values.ravel(), XGBoost__eval_set=eval_set, XGBoost__eval_metric=r2_score)